In [1]:
import tensorflow as tf

In [9]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable(
    "v",initializer=tf.zeros_initializer()(shape=[1]))
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable(
    "v",initializer=tf.ones_initializer()(shape=[1]))
    
# 书上的最后一行的zeros_initializer是没有加括号的，会出bug，加括号就好了

In [8]:
with tf.Session(graph = g1) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("",reuse = True):
        print(sess.run(tf.get_variable("v")))

[0.]


In [10]:
with tf.Session(graph = g2) as sess:
    tf.initialize_all_variables().run()
    with tf.variable_scope("",reuse = True):
        print(sess.run(tf.get_variable("v")))

[1.]


In [11]:
a = tf.constant([1.0,2.0],name="a")
b = tf.constant([2.0,3.0],name="b")
result = tf.add(a,b,name="add")
print(result)

Tensor("add:0", shape=(2,), dtype=float32)


In [12]:
tf.Session().run(result)

array([3., 5.], dtype=float32)

In [13]:
sess = tf.Session()
with sess.as_default():
    print(result.eval())
    

[3. 5.]


In [17]:

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.constant([[0.7,0.9]])

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()
sess.run(w1.initializer)
sess.run(w2.initializer)

print(sess.run(y))
sess.close()

[[3.957578]]


In [21]:
# 通过placeholder实现前向传播算法

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(1,2),name="input")
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

sess = tf.Session()
init_op = tf.initialize_all_variables()
sess.run(init_op)

print(sess.run(y,feed_dict={x:[[0.7,0.9]]}))

# 和上面的差别倒也没多大，主要就是这个place_holder，至于后面的init_op，都可以这么用

[[3.957578]]


In [24]:
# 完整的神经网络样例 二分类

from numpy.random import RandomState

batch_size = 8

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_ = tf.placeholder(tf.float32,shape=(None,1),name='y-input')

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

cross_entropy = -tf.reduce_mean(
                y_ * tf.log(tf.clip_by_value(y,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size,2)

Y = [[int(x1+x2 < 1)] for (x1,x2) in X]

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    
    sess.run(init_op)
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 5000
    
    for i in range(STEPS):
        
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        sess.run(train_step,
                feed_dict={x:X[start:end],y_:Y[start:end]})
        
        if i % 1000 ==0:
            total_cross_entropy = sess.run(
                cross_entropy,feed_dict={x:X,y_:Y})
            print('After %d training steps,cross entropy on all data is %g'
                 % (i,total_cross_entropy))

    print(sess.run(w1))
    print(sess.run(w2))
          

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training steps,cross entropy on all data is 0.0674925
After 1000 training steps,cross entropy on all data is 0.0163385
After 2000 training steps,cross entropy on all data is 0.00907547
After 3000 training steps,cross entropy on all data is 0.00714436
After 4000 training steps,cross entropy on all data is 0.00578471
[[-1.9618275  2.582354   1.6820377]
 [-3.4681718  1.0698231  2.11789  ]]
[[-1.824715 ]
 [ 2.6854665]
 [ 1.418195 ]]


In [ ]:
'''
所有的神经网络训练都可以分为3个步骤：
1. 定义神经网络的结构和前向传播的输出算法
2. 定义损失函数以及选择反向传播算法
3. 生成会话并且在训练数据上反复运行反向传播优化算法
'''